# PoR test analysis

PoR tests are mostly defined over the single server scenario and assuming an idealized large block code, i.e. a code where a dataset is split into a large number of chunks (`K`) and encoded to an even larger number of chunks (`N`). Most theoretic results on the PoR tests are derived with these assumptions.

In our use case, however, we should analyze it under more realistic assumptions, meaning:
 - the erasure code is limited to smaller block sizes (i.e. `N` and `K` are limited and the number of chunks can be higher than these
 - the dataset is spread over mulitiple nodes.

The coding and the distribution of chunks over multiple nodes might be orchestrated in many ways. Factors influencing the true/false positive/negative outcome of the test include:
- the selection of erasure coding parameters,
- chunk placement strategy,
- test query size,
- the selection of chunks for the tests,
- etc.

In what follows we analyze the test outcome under various assumptions. 3 types of analysis will be provided (WIP)
- static analysis under random erasure,
- dynamic analysis under random erasure,
- analysis under adversarial erasure.


# Reference Model

Our reference model assumes the upload of a dataset as follows. The dataset is first split to blocks and each block is erasure coded. Assuming a systematic code, data chunks and parity chunks are the outcome of the erasure coding. Each of these chunks are then tagged using PoR tagging.

Assuming a general structure that can accomodate various dataset sizes, block sizes, erasure code limitations and chunk size constraints, we can assume the following structure.

A dataset of size `F` bytes is split into `B` blocks. Each block is individually erasure coded using an `[N,K]` erasure code, using a symbols size of `F/B/K`.

*(Note: more complex multi-layer coding schemes are also possible, but left for later discussion)*

Resulting erasure symbols are then indivually PoR tagged.
PoR tagging, in it's base form using the BLS12-381 curve, operates on `Z=31` byte symbols, also called the sectors in PoR literature. However, this can be extended to larger sizes in two ways.
- First, the PoR scheme has its own blocks, governed by the parameter `S`. `S`, the PoR block size, determines how large is the tagging overhead (inversely proportional to `S`, and also the size of the proofs (approx. `S+1` symbols),
- Second, nothing prevents us from splitting the chunk into multiple (`T`) PoR units and apply tagging on each of these `T` units individually, handling the resulting tag as a "vector tag".

Thus, while the erasure coded symbol contains  `F/B/K/Z` PoR symbols, the erasure code symbol size and the PoR block size does not have to be the same. To give an example, say `F/B/K = 3100` bytes is our chunk size, which is 100 PoR symbols. Resulting tag and proof sizes are shown below:

| S | T | tag size (48T) bytes | tag overhead | proof size (32*S+48) bytes |
| --- | --- | --- | --- | --- |
|100|  1| 48   |  1.5%| 3248 |
| 10| 10| 480  | 15%  |  368 |
|  5| 20| 960  | 30%  |  208 |
|  1|100| 4800 |150%  |   80 |
- tags are on the G_1 curve, thus take 48 bytes in compressed form per symbol
- proofs contain a linear combination of data blocks and a tag

## Notebook imports

In [ ]:
from scipy.stats import hypergeom, binom
import numpy as np
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [15, 5]

# Non adversarial model (static)

We assume 
- random chunk losses, and 
- random node outages.

We evaluate 
- whether the file is recoverable
- the probability of a test passing


## Random chunk erasure

Let's first start with independent random chunk losses and random queries.
We study the probabilities of
- still having the data recoverable
- passing or failing various tests

as a function of system parameters, dataset size, and the assumed chunk loss probability. 

We define two types of tests:
- a simple random query, verifying $l$ chunks, passing only if all of these are available;
- a multiquery, composed of $q$ simple queries, passing if at most *maxfail* of the $q$ queries fail.

In [ ]:
def P_recoverable(p_chunk_error, n, k, b):
  ## Calculate_probability of recoverable file
  ## assuming independent chunk failures in any given ECC block
  e = n-k+1 # erased chunks meaning loss
  P_block_pass = binom(n,p_chunk_error).cdf(e-1) #prob. less than e chunks fail in a block
  return P_block_pass ** b

def P_pass_random_query(p_chunk_error, l):
  ## Calculate_probability of a random query failing
  ## assuming independent chunk losses
  P_chunk_pass = 1 - p_chunk_error #prob. less than e chunks fail in a block
  return P_chunk_pass ** l

def P_pass_random_multiquery(p_chunk_error, l, q=1, maxfail=0):
  ## Calculate_probability of at most maxfail of q random queries failing
  ## The test is passed if more than q-maxfail individual queries pass.
  ## assuming independent chunk losses
  P_chunk_pass = 1 - p_chunk_error #prob. less than e chunks fail in a block
  P_query_pass = P_chunk_pass ** l
  P_multiquery_pass = binom(q, 1-P_query_pass).cdf(maxfail)
  return P_multiquery_pass


To verify the above functions we set up a small simulation, and evaluate the same probabilities empirically as well.
In this case we also allow for an optional additional layer of erasure coding "horizontally" on the row, i.e. among chunks on the
same node.

In [ ]:
def random_chunks(p_chunk_error):
  #chunks = np.full((N, B), True)
  chunks = np.random.rand(N, B)
  return chunks > p_chunk_error

def recoverable(chunks, ecol, erow=0):
  # ecol: maximum number of recoverable errors in a column
  # erow: maximum number of recoverable errors in a row
  badcol = chunks.shape[0] - np.count_nonzero(chunks, axis=0) # bad chunks per column
  badrow = chunks.shape[1] - np.count_nonzero(chunks, axis=1)
  return not (chunks[badrow > erow][:, badcol > ecol]).any()

def Px_recoverable(p_chunk_error, n, k, mcol=0, tries=1000, random_chunks=random_chunks):
  ## evaluate probability of dataset recoverable empirically
  e = n-k # erased chunks meaning loss
  count = 0 
  for i in range(tries):
    count += recoverable(random_chunks(p_chunk_error), e, mcol)
  return count/tries

def random_query(chunks, l):
  rng = np.random.default_rng()
  return rng.choice(chunks.flatten(), l, axis=0, replace=False).all()

def Px_pass_random_query_on_chunks(chunks, l, tries = 100):
  ## evaluate probability of passing test empirically
  count = 0 
  for i in range(tries):
    count += random_query(chunks,l)
  return count/tries

def Px_pass_random_query(p_chunk_error, l, tries = 1000):
  ## evaluate probability of passing test empirically
  count = 0 
  for i in range(tries):
    count += random_query(random_chunks(p_chunk_error),l)
  return count/tries

def random_multiquery(chunks, l, q=1, maxfail=0):
  count = 0 # number of queries passed
  for i in range(q):
    count += random_query(chunks, l)
  return q-count <= maxfail

def Px_pass_random_multiquery(p_chunk_error, l, q=1, maxfail=0, tries = 1000):
  ## evaluate probability of passing test empirically
  count = 0 
  for i in range(tries):
    count += random_multiquery(random_chunks(p_chunk_error), l, q, maxfail)
  return count/tries

def Stats_random_query_on_chunks(p_chunk_error, n, k, l, tries = 1000):
  ## TP: recoverable and test for rec. passed
  e = n-k # maximum number of errors in a column
  tp, fp, tn, fn = 0, 0, 0, 0 
  for i in range(tries):
    chunks = random_chunks(p_chunk_error)
    rec = recoverable(chunks, e)
    testrec = random_query(chunks,l)
    tp += rec and testrec
    fp += rec and not testrec
    tn += not rec and not testrec
    fn += not rec and testrec
  return (tp/tries, fn/tries, fp/tries, tn/tries)

def Stats_random_multiquery_on_chunks(p_chunk_error, n, k, l, q=1, maxfail=0, tries = 1000):
  ## TP: recoverable and test for rec. passed
  e = n-k # maximum number of errors in a column
  tp, fp, tn, fn = 0, 0, 0, 0 
  for i in range(tries):
    chunks = random_chunks(p_chunk_error)
    rec = recoverable(chunks, e)
    testrec = random_multiquery(chunks, l, q, maxfail)
    tp += rec and testrec
    fn += rec and not testrec
    tn += not rec and not testrec
    fp += not rec and testrec
  return (tp/tries, fn/tries, fp/tries, tn/tries)

In [ ]:
#@title Encoding and Query Parameters

#@markdown block code K
K = 50 #@param {type:"slider", min:1, max:100, step:1}
#@markdown block code N
N = 100 #@param {type:"slider", min:1, max:300, step:1}
#@markdown query length
L = 10 #@param {type:"slider", min:1, max:100, step:1}
#@markdown number of blocks (chunks per node)
B = 100 #@param {type:"slider", min:1, max:100, step:1}

F = K*B # chunks 

p_chunk_error = .34
print("# Evaluating probabilities both analytical and empirical")
print("Prob file recoverable:",
      P_recoverable(p_chunk_error, N, K, B),
      Px_recoverable(p_chunk_error, N, K))
print("Prob random query passed:",
      P_pass_random_query(p_chunk_error, L),
      Px_pass_random_query(p_chunk_error, L))
print("Stats (TP, FN, FP, TN):",
      Stats_random_query_on_chunks(p_chunk_error, N, K, L))

p_chunk_error = .14
Q=10
MAXFAIL=9
print("Prob random multiquery passed:",
      P_pass_random_multiquery(p_chunk_error, L, Q, MAXFAIL),
      Px_pass_random_multiquery(p_chunk_error, L, Q, MAXFAIL))
print("Stats (TP, FN, FP, TN):",
      Stats_random_multiquery_on_chunks(p_chunk_error, N, K, L, Q, MAXFAIL))



Next, we plot these with a given set of system parameters as a funtion of chunk loss probability. This on one hand verifies that the analytic and empirical results match, on the other hand shows what to expect from tests in different system states.

In [ ]:
def plotall(X, xlabel, N, K, B, L, P_chunk_error):
  fig = plt.figure()
  ax = fig.add_subplot(111)
  ax.plot(X, np.vectorize(Px_recoverable)(P_chunk_error, N, K, B), label="Emp. Prob file recoverable")
  ax.plot(X, np.vectorize(P_recoverable)(P_chunk_error, N, K, B), label="Prob file recoverable")
  ax.plot(X, np.vectorize(Px_pass_random_query)(P_chunk_error, L), label="Emp. Prob random query passed")
  ax.plot(X, np.vectorize(P_pass_random_query)(P_chunk_error, L), label="Prob random query passed")
  ax.plot(X, np.vectorize(Px_pass_random_multiquery)(P_chunk_error, L, Q, MAXFAIL), label="Emp. Prob random multiquery (9/10) passed")
  ax.plot(X, np.vectorize(P_pass_random_multiquery)(P_chunk_error, L, Q, MAXFAIL), label="Prob random multiquery (9/10) passed")
  ax.set_xlabel(xlabel)
  ax.set_ylabel('probability')
  ax.legend()
  plt.show()

def printplotall(X, xlabel, N, K, B, L, P_chunk_error):
  def checkprint(v, label):
    if not isinstance(v, np.ndarray):
      print(label, v)
  checkprint(N, "block code N:")
  checkprint(K, "block code K:")
  checkprint(B, "chunks per node:")
  checkprint(L, "query length:")
  plotall(X, xlabel, N, K, B, L, P_chunk_error)

K = 50 # block code k
N = 100 # block code n
L = 5 # query size in chunks
F = 5000 # file size in chunks
P_chunk_error = np.arange(0,1,.01) 
Q=10
MAXFAIL=9

B = int(F/K) # chunks per node

printplotall(P_chunk_error, "chunk failure probability", N, K, B, L, P_chunk_error)


As expected, empirical and analytical curves overlap.

Assuming random losses, whether the file is recoverable is close to a step function of chunk loss probability, which is a behavior we prefer as it simplifies estimating system state. 

A simple random query as a test (at least with these parameters) is clearly not a good indicator of dataset loss.

The multi-query is much more promising, although clearly there are problems of accuracy.

In the next plot, we look more into the details of the information contained in the outputs of a multi-query. `Q=10` queries (each challenging `L=5` chunks) are performed, and the test is passed if at most `maxfail` of these fails.

In [ ]:
X, xlabel = P_chunk_error, "chunk failure probability"
fig = plt.figure()
ax = fig.add_subplot(111)
ax.plot(X, np.vectorize(P_recoverable)(P_chunk_error, N, K, B), label="Prob file recoverable")
ax.plot(X, np.vectorize(P_pass_random_multiquery)(P_chunk_error, L, 10, 1), label="Prob random multiquery (1/10) passed")
ax.plot(X, np.vectorize(P_pass_random_multiquery)(P_chunk_error, L, 10, 6), label="Prob random multiquery (6/10) passed")
ax.plot(X, np.vectorize(P_pass_random_multiquery)(P_chunk_error, L, 10, 7), label="Prob random multiquery (7/10) passed")
ax.plot(X, np.vectorize(P_pass_random_multiquery)(P_chunk_error, L, 10, 8), label="Prob random multiquery (8/10) passed")
ax.plot(X, np.vectorize(P_pass_random_multiquery)(P_chunk_error, L, 10, 9), label="Prob random multiquery (9/10) passed")
ax.set_xlabel("chunk failure probability")
ax.set_ylabel('probability')
ax.legend()
plt.show()

ax.set_yscale('log')
ax.set_ylim(1e-4, 1)
fig


In [ ]:
X, xlabel = P_chunk_error, "chunk failure probability"
fig = plt.figure()
ax = fig.add_subplot(111)
ax.plot(X, np.vectorize(P_recoverable)(P_chunk_error, N, K, B), label="Prob file recoverable")
ax.plot(X, np.vectorize(P_pass_random_query)(P_chunk_error, 50), label="Prob random query L=50 passed")
#ax.plot(X, np.vectorize(P_pass_random_multiquery)(P_chunk_error, 50, 1, 0), label="Prob random multiquery L=50 (1/0) passed")
ax.plot(X, np.vectorize(P_pass_random_multiquery)(P_chunk_error, 5, 10, 9), label="Prob random multiquery L=5 (5/10) passed")
ax.plot(X, np.vectorize(P_pass_random_multiquery)(P_chunk_error, 1, 50, 19), label="Prob random multiquery L=1 (25/50) passed")
ax.set_xlabel("chunk failure probability")
ax.set_ylabel('probability')
ax.legend()
plt.show()

ax.set_yscale('log')
ax.set_ylim(1e-4, 1)
fig


### Test Accuracy

Time to analyze test outcomes from the perspective of True/False Positive/Negative results.

In [ ]:
from functools import partial

def plotall(X, xlabel, N, K, B, L, P_chunk_error):
  fig = plt.figure()
  ax = fig.add_subplot(111)
  #stats = np.vectorize(Stats_random_multiquery_on_chunks)(p_chunk_error, N, K, L, Q, MAXFAIL)
  ax.plot(X, np.vectorize(P_recoverable)(P_chunk_error, N, K, B), label="Prob file recoverable")
  stats = np.vectorize(partial(Stats_random_multiquery_on_chunks, n=N, k=K, l=L, q=Q, maxfail=MAXFAIL))(P_chunk_error)
  ax.stackplot(X, stats, labels=['TP', 'FN', 'FP', 'TN'])
  ax.set_xlabel(xlabel)
  ax.set_ylabel('rate')
  ax.legend()
  plt.show()

def printplotall(X, xlabel, N, K, B, L, P_chunk_error):
  def checkprint(v, label):
    if not isinstance(v, np.ndarray):
      print(label, v)
  checkprint(N, "block code N:")
  checkprint(K, "block code K:")
  checkprint(B, "chunks per node:")
  checkprint(L, "query length:")
  checkprint(Q, "multi-query length:")
  checkprint(MAXFAIL, "multi-query max fail:")
  plotall(X, xlabel, N, K, B, L, P_chunk_error)


In [ ]:
K = 50 # block code k
N = 100 # block code n
L = 5 # query size in chunks
F = 5000 # file size in chunks
P_chunk_error = np.arange(0,1,.05) 
Q=10
MAXFAIL=9
B = int(F/K) # chunks per node

printplotall(P_chunk_error, "chunk failure probability", N, K, B, L, P_chunk_error)

As a reminder, in the above plot:
- TP: recoverable and test passed
- FN: recoverable, but test failed. So we think it is lost, while it is not 
- FP: not recoverable, but test passed. So we don't know it is already lost
- TN: not recoverable, and test showed it


In [ ]:
MAXFAIL=7
printplotall(P_chunk_error, "chunk failure probability", N, K, B, L, P_chunk_error)

In [ ]:
F = 500000 # file size in chunks
B = int(F/K) # chunks per node
MAXFAIL=9
printplotall(P_chunk_error, f'chunk failure probability (B={B})', N, K, B, L, P_chunk_error)

In [ ]:
F = 250 # file size in chunks
B = int(F/K) # chunks per node
MAXFAIL=9
printplotall(P_chunk_error, f'chunk failure probability (B={B})', N, K, B, L, P_chunk_error)

## Random node failures

Let's focus our attention on node erasures and outages. There are two types of node issues:
 - node erasures, which are permanent losses
 - node outages, which are temporary

It is the usualy assumption that a test cannot differentate between the two. Yet, both exist and their effects are different. Outages add noise to the system, but do not provoke a real loss of the data. Durability is not affected by outages. Node erasures, instead, provoke data loss just as individual chunk losses.

Model:
 - p_node_outage
 - p_node_erasure

Naive approach: just redefine the random_chunks function to remove entire rows

In [ ]:
def random_chunks(p_node_error):
  #chunks = np.full((N, B), True)
  #chunks = np.random.rand(N, B)
  nodes = np.random.rand(N)
  nodes = nodes > p_node_error
  chunks = np.outer(nodes, np.full((B), True))
  return chunks


In [ ]:
N=10
B=6
random_chunks(.3)

In [ ]:
K = 50 # block code k
N = 100 # block code n
L = 1 # query size in chunks
F = 50 # file size in chunks
P_chunk_error = np.arange(0,1,.05) 
Q=10
MAXFAIL=9
B = int(F/K) # chunks per node

printplotall(P_chunk_error, "chunk failure probability", N, K, B, L, P_chunk_error)


In [ ]:
MAXFAIL=7
printplotall(P_chunk_error, "chunk failure probability", N, K, B, L, P_chunk_error)

In [ ]:
# F = 50 # file size in chunks
# B = int(F/K) # chunks per node
# MAXFAIL=9
# printplotall(P_chunk_error, f'chunk failure probability (B={B})', N, K, B, L, P_chunk_error)

In [ ]:
# L = 1 # query size in chunks
# printplotall(P_chunk_error, f'chunk failure probability (B={B})', N, K, B, L, P_chunk_error)

In [ ]:
MAXFAIL=6
printplotall(P_chunk_error, f'chunk failure probability (B={B})', N, K, B, L, P_chunk_error)

In [ ]:
def random_chunks(p_node_error):
  #chunks = np.full((N, B), True)
  #chunks = np.random.rand(N, B)
  nodesfail = np.random.choice(N, round(p_node_error*N),replace=False )
  nodes = np.full((N), True)
  for i in nodesfail:
    nodes[i] = False
  chunks = np.outer(nodes, np.full((B), True))
  return chunks

In [ ]:
N=10
B=6
random_chunks(.3)

In [ ]:
K = 50 # block code k
N = 100 # block code n
L = 1 # query size in chunks
F = 50 # file size in chunks
P_chunk_error = np.arange(0,1,.05) 
Q=10
MAXFAIL=6
B = int(F/K) # chunks per node
printplotall(P_chunk_error, f'chunk failure probability (B={B})', N, K, B, L, P_chunk_error)

In [ ]:
MAXFAIL=5
printplotall(P_chunk_error, f'chunk failure probability (B={B})', N, K, B, L, P_chunk_error)

In [ ]:
Q=20
MAXFAIL=10
printplotall(P_chunk_error, f'chunk failure probability (B={B})', N, K, B, L, P_chunk_error)

In [ ]:
K = 50 # block code k
N = 100 # block code n
L = 1 # query size in chunks
F = 50 # file size in chunks
P_chunk_error = np.arange(0,1,.05) 
Q=10
MAXFAIL=6
B = int(F/K) # chunks per node
printplotall(P_chunk_error, f'chunk failure probability (B={B})', N, K, B, L, P_chunk_error)

# Adversarial erasure model

A peculiarity of our trustless systems compared to the application of erasure codes and PoR in a datacenter scenario is that erasures could be adversarial. An adversary (or adversaries) can use their knowledge about the encoding and verification to cherry-pick chunks to minimize the chances of getting discovered while maximizing harm.

Assuming an [N,K] Reed-Solomon erasure coding, the erasure of `N-K+1` chunks from a block makes that block unrecoverable. In the adversarial scenario, each other block can remain intact, i.e. the removal of `N-K-1` selected chunks from the overall `N*B` chunks makes the file unrecoverable.

In [ ]:
K = 50 # block code k
N = 100 # block code n
L = 100 # query size in chunks
F = 5000 # file size in chunks
B = F/K # chunks per node
C = N*B # total chunks

# ADVERSARIAL assumption: n-k+1 chunks erased from one ECC block
E = N-K+1 # erased chunks

# random query over all n*b chunks WITH replacement (assume l << n*b)
def P_pass_random(e, c, l):
  return (1 - e/c) ** l  
print("random:", P_pass_random(E, C, L))

# random query over all n*b chunks WITH replacement, node first (assume l << n*b)
# probability of selecting node without lost chunk: ((n-e)/n)
# probability of selecting node with lost chunk: (e/n)
def P_pass_random2(n, e, b, l):
   return ((n-e)/n + e/n * (b-1)/b) ** l
print("random:", P_pass_random2(N, E ,B ,L))

# random query over all n*b chunks WITHOUT replacement
def P_pass_random_wor(e, c, l):
  return hypergeom(c, e, l).pmf(0)  
print("random w/o replacement:", P_pass_random_wor(E, C, L))

def P_pass_random_wor2(n, e, b, l):
  c = n*b
  return P_pass_random_wor(e, c, l)

# hit L nodes, query 1 random chunk from each (assume l <= n)
#sum p(k lines with error selected) * (b-1)/b ** k
def P_pass_pernode_random(n, e, b, l):
  ret = 0
  for i in np.arange(0, e+1):
    ret += hypergeom(n, e, l).pmf(i) * ((b-1)/b) ** i
  return ret
print("L nodes, 1 random chunk each:", P_pass_pernode_random(N, E, B, L))

#P_pass_pernode_random = (n-e)/n + e/n * (b-1)/b
def P_pass_pernode_random(n, e, b, l):
  return ((n-l)/n + l/n*(1 - 1/b)) ** e # for nodes with no error, pass. For the e nodes with single error, pass with 1-1/b, independent per node
print("L nodes, 1 random chunk each (approx):", P_pass_pernode_random(N, E, B, L))

# hit L nodes, query same chunk (assume l <= n)
# P(good chunk) + P(bad chunk) * P(least 1 bad node | bad chunk)
def P_pass_pernode_same(n, e, b, l):
  return np.where(
      l <= n,
      (b-1)/b + 1/b * ((n-l)/n) ** l,
      np.NaN)
print("L nodes, same chunk each:", P_pass_pernode_same(N, E, B, L))

# select random node, query random chunks from it (assume b >= l)
def P_pass_samenode_random(n, e, b, l):
  return np.where(
      b >= l,
      (n-e)/n  + (e/n) * (1-l/b) ** l,
      np.NaN)
print("random node, L chunks:",P_pass_samenode_random(N, E, B, L))

# select random node, query random chunks from it (assume b >= l)
def P_pass_samenode_random_wor(n, e, b, l):
  return np.where(
      b >= l,
      (n-e)/n  + (e/n) * hypergeom(b, 1, l).pmf(0),
      np.NaN)
print("random node, L chunks w/o repl:",P_pass_samenode_random_wor(N, E, B, L))


In [ ]:
def plotall(title, X, xlabel, N, E, B, L):
  fig = plt.figure()
  ax = fig.add_subplot(111)
  #ax.plot(X, 1 - P_pass_random2(N, E, B, L), 'm', label="random2")
  ax.plot(X, 1 - P_pass_random_wor2(N, E, B, L), 'c', label="random_wor")
  ax.plot(X, 1 - P_pass_pernode_random(N, E, B, L), 'k', label="pernode_random")
  ax.plot(X, 1 - P_pass_pernode_same(N, E, B, L), 'g', label="pernode_same")
  #ax.plot(X, 1 - P_pass_samenode_random(N, E, B, L), 'b', label="samenode_random")
  ax.plot(X, 1 - P_pass_samenode_random_wor(N, E, B, L), 'r', label="samenode_random_wor")
  ax.set_xlabel(xlabel)
  ax.set_ylabel('probability of detecting adv. erasure')
  ax.legend()
  fig.suptitle(title)
  plt.show()

def printplotall(title, X, xlabel, N, E, B, L):
  def checkprint(v, label):
    if not isinstance(v, np.ndarray):
      print(label, v)
  checkprint(N, "block code N:")
  checkprint(E, "block code N-K+1:")
  checkprint(B, "chunks per node:")
  checkprint(L, "query length:")
  plotall(title, X, xlabel, N, E, B, L)

K = 50 # block code k
N = 100 # block code n
L = 40 # query size in chunks
F = 500000 # file size in chunks
B = F/K # chunks per node
# ADVERSARIAL assumption: n-k+1 chunks erased from one ECC block
E = N-K+1 # erased chunks

L = np.arange(1,200) # query size in chunks
printplotall("Test against adv. erasure, as a function of query size",
    L, "query size in chunks", N, E, B, L)
L = 10 # query size in chunks

F = np.arange(50, 50000, 100) # file size in chunks
B = F/K # chunks per node
printplotall("Test against adv. erasure, as a function of number of erasure blocks",
    B, "chunks per node", N, E, B, L)
F = 5000 # file size in chunks
B = F/K # chunks per node

N = np.arange(50, 500) # block code n
E = N-K+1 # erased chunks
printplotall("Test against adv. erasure, as a function of code rate",
    N, "block code 'n'", N, E, B, L)
N = 10 # block code n
E = N-K+1 # erased chunks